In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from openai import OpenAI
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
openai = OpenAI(
    api_key= os.getenv('kurtim_api_key'), # Refer to Create a secret key section
    base_url="https://cloud.olakrutrim.com/v1",
)

In [3]:
vectors = FAISS.load_local("../VectorStores/nomic/AllPrinters", OllamaEmbeddings(model =  "nomic-embed-text"), allow_dangerous_deserialization= True)
retriever = vectors.as_retriever()

In [4]:
def multiQueryRetrieval(question):

    systemMessage = """You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines. Do not add any further lines in the response except for the rephrased questions"""

    chat_completion = openai.chat.completions.create(
    model="Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "system", "content": systemMessage},
        {"role": "user", "content": question}
    ],
    logit_bias= {2435: -100, 640: -100},
    max_tokens= 2000,
    temperature= 0, # Optional, Defaults to 1. Range: 0 to 2
    top_p= 1 # Optional, Defaults to 1. It is generally recommended to alter this or temperature but not both.
    )

    response = chat_completion.choices[0].message.content

    queries = response.split('\n\n')
    
    retrievedChunks = []
    for sentence in queries:
        chunks = retriever.invoke(sentence)
        for chunk in chunks:
            retrievedChunks.append(chunk)
    sortedArray = []
    for doc in retrievedChunks:
        if doc not in sortedArray:
            sortedArray.append(doc)

    return sortedArray


In [5]:
def combineDocs(docs):
    context = "\n\n".join(f'Document [Metadata :  Source : {doc.metadata.get("source")} \nContent : \n{doc.page_content} ]' for doc in docs)
    return context

In [6]:
chatHistory = ["","","","","",""]

def chat(question): 
    
    global chatHistory
    contextString = combineDocs(multiQueryRetrieval(question))
    
    chat_completion = openai.chat.completions.create(
    model="Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant. You are given conversation history, context and a question from the user. Answer the question based on the information given in the text or based on the conversation if needed"},
        {"role": "user", "content": chatHistory[-6]},
        {"role": "assistant", "content": chatHistory[-5]},
        {"role": "user", "content": chatHistory[-4]},
        {"role": "assistant", "content": chatHistory[-3]},
        {"role": "user", "content": chatHistory[-2]},
        {"role": "assistant", "content": chatHistory[-1]},
        {"role": "user", "content": "Here is some context : \n" + contextString},
        {"role": "user", "content": question}
    ],
    logit_bias= {2435: -100, 640: -100},
    max_tokens= 2000,
    temperature= 0, # Optional, Defaults to 1. Range: 0 to 2
    top_p= 1 # Optional, Defaults to 1. It is generally recommended to alter this or temperature but not both.
    )

    response = chat_completion.choices[0].message.content

    chatHistory.append(question)
    chatHistory.append(response)

    print(response)
    
    return response


In [8]:
chat("what are the various parts of a printer in a Deskjet3700")

Based on the provided documents, specifically the HP DeskJet 3700 All-in-One series document, the technical information section does not provide a detailed list of the various parts of a printer. However, I can suggest some common parts found in a typical printer:

1. Paper Tray: Holds the paper to be printed.
2. Print Head: Responsible for printing the ink onto the paper.
3. Ink Cartridges: Contain the ink used for printing.
4. Printhead Assembly: Holds the print head and ink cartridges.
5. Paper Feed Mechanism: Moves the paper through the printer.
6. Fuser Unit: Fuses the toner to the paper in laser printers.
7. Toner Cartridge: Contains the toner used for printing.
8. Control Panel: The interface where you can select print options, check print status, and perform other functions.
9. Power Supply: Provides power to the printer.
10. Motor and Fans: Help move the paper and other components within the printer.
11. Pickup, Feed, and Delivery Mechanism: Moves the paper through the printer

'Based on the provided documents, specifically the HP DeskJet 3700 All-in-One series document, the technical information section does not provide a detailed list of the various parts of a printer. However, I can suggest some common parts found in a typical printer:\n\n1. Paper Tray: Holds the paper to be printed.\n2. Print Head: Responsible for printing the ink onto the paper.\n3. Ink Cartridges: Contain the ink used for printing.\n4. Printhead Assembly: Holds the print head and ink cartridges.\n5. Paper Feed Mechanism: Moves the paper through the printer.\n6. Fuser Unit: Fuses the toner to the paper in laser printers.\n7. Toner Cartridge: Contains the toner used for printing.\n8. Control Panel: The interface where you can select print options, check print status, and perform other functions.\n9. Power Supply: Provides power to the printer.\n10. Motor and Fans: Help move the paper and other components within the printer.\n11. Pickup, Feed, and Delivery Mechanism: Moves the paper throug

In [9]:
chat("what are the steps to print a photo on photo paper (Windows) using deskjet 2623")

Based on the provided documents, here are the steps to print a photo on photo paper (Windows) using the HP Deskjet 2623:

1. From your software, select Print.
2. Make sure your printer is selected.
3. Click the button that opens the Properties dialog box. Depending on your software application, this button might be called Properties, Options, Printer Setup, Printer Properties, Printer, or Preferences.
4. Select the appropriate options on the Printing Shortcuts or Printing Shortcut tab.
5. Click OK to return to the Properties dialog box.
6. Click OK, and then click Print or OK in the Print dialog box.

Additionally, you can also follow these steps:

* Before printing, make sure you have photo paper loaded in the input tray and the output tray is open. For more information about loading paper, see Load media on page 11.
* Choose the appropriate paper size in the Paper Size pop-up menu. NOTE: If you change the paper size, ensure that you have loaded the correct paper.
* Select the orienta

'Based on the provided documents, here are the steps to print a photo on photo paper (Windows) using the HP Deskjet 2623:\n\n1. From your software, select Print.\n2. Make sure your printer is selected.\n3. Click the button that opens the Properties dialog box. Depending on your software application, this button might be called Properties, Options, Printer Setup, Printer Properties, Printer, or Preferences.\n4. Select the appropriate options on the Printing Shortcuts or Printing Shortcut tab.\n5. Click OK to return to the Properties dialog box.\n6. Click OK, and then click Print or OK in the Print dialog box.\n\nAdditionally, you can also follow these steps:\n\n* Before printing, make sure you have photo paper loaded in the input tray and the output tray is open. For more information about loading paper, see Load media on page 11.\n* Choose the appropriate paper size in the Paper Size pop-up menu. NOTE: If you change the paper size, ensure that you have loaded the correct paper.\n* Sele

In [10]:
chat("How to setup web services in HP deskjet 2600?")

BadRequestError: Bad Request

In [ ]:
chat("How to load paper in HP Deskjet 4100e?")

Based on the provided documents, the steps to load paper in HP Deskjet 4100e are:

1. Raise the input tray and input tray extender.
2. Insert a stack of paper into the input tray with the short edge down and the print side up, and slide the stack of paper down until it stops.
3. Slide the paper-width guide to the right until it stops at the edge of the paper.

Note: These steps are based on the HP Deskjet-3700-Printer.pdf document, which is likely to be applicable to the HP Deskjet 4100e as well.


'Based on the provided documents, the steps to load paper in HP Deskjet 4100e are:\n\n1. Raise the input tray and input tray extender.\n2. Insert a stack of paper into the input tray with the short edge down and the print side up, and slide the stack of paper down until it stops.\n3. Slide the paper-width guide to the right until it stops at the edge of the paper.\n\nNote: These steps are based on the HP Deskjet-3700-Printer.pdf document, which is likely to be applicable to the HP Deskjet 4100e as well.'

In [ ]:
chat("Where does paper go in HP Deskjet 4100e?")

According to the provided documents, the paper goes in the input tray of the HP Deskjet 4100e. To load paper, follow these steps:

1. Raise the input tray.
2. Slide the paper width guide all the way to the left.
3. Load a stack of plain white paper into the input tray.
4. Slide the paper width guide to the right until it rests at the edge of the paper.
5. Lower the output tray, and then pull out the tray extender.

Note: The paper should be loaded with the print side down.


'According to the provided documents, the paper goes in the input tray of the HP Deskjet 4100e. To load paper, follow these steps:\n\n1. Raise the input tray.\n2. Slide the paper width guide all the way to the left.\n3. Load a stack of plain white paper into the input tray.\n4. Slide the paper width guide to the right until it rests at the edge of the paper.\n5. Lower the output tray, and then pull out the tray extender.\n\nNote: The paper should be loaded with the print side down.'

In [ ]:
chat("How to automatically print on both sides of the paper in HP e877 series?")

According to the provided documents, to automatically print on both sides of the paper in the HP e877 series, follow these steps:

1. From the software program, select the Print option.
2. Select the printer from the list of printers, and then click the Printer Properties or Preferences button to open the print driver.
3. Click the Finishing tab.
4. Select Print on both sides. Click OK to close the Document Properties dialog.
5. In the Print dialog, click Print to print the job.

Note: Make sure your printer has an automatic duplexer installed and that the paper size and type are supported by the duplexer.


'According to the provided documents, to automatically print on both sides of the paper in the HP e877 series, follow these steps:\n\n1. From the software program, select the Print option.\n2. Select the printer from the list of printers, and then click the Printer Properties or Preferences button to open the print driver.\n3. Click the Finishing tab.\n4. Select Print on both sides. Click OK to close the Document Properties dialog.\n5. In the Print dialog, click Print to print the job.\n\nNote: Make sure your printer has an automatic duplexer installed and that the paper size and type are supported by the duplexer.'

In [ ]:
chat("What is the contact number for ordering supplies in Denmark?")

According to the provided documents, there is no specific contact number mentioned for ordering supplies in Denmark. However, you can order original HP supplies for the HP DeskJet 2600 series by going to www.hp.com/buy/supplies and following the prompts to find the right cartridges for your printer.


'According to the provided documents, there is no specific contact number mentioned for ordering supplies in Denmark. However, you can order original HP supplies for the HP DeskJet 2600 series by going to www.hp.com/buy/supplies and following the prompts to find the right cartridges for your printer.'

In [ ]:
chat("How to check the ink levels for HP deskjet 2600 on a mac os machine?")

According to the provided documents, to check the ink levels for HP Deskjet 2600 on a Mac OS machine, follow these steps:

1. Open HP Utility.
2. Select the HP DeskJet 2600 series from the list of devices at the left of the window.
3. Click Supplies Info.
4. The print cartridge ordering numbers appear.
5. Click All Settings to return to the Information and Support pane.

Alternatively, you can also check the ink levels from the printer control panel.


'According to the provided documents, to check the ink levels for HP Deskjet 2600 on a Mac OS machine, follow these steps:\n\n1. Open HP Utility.\n2. Select the HP DeskJet 2600 series from the list of devices at the left of the window.\n3. Click Supplies Info.\n4. The print cartridge ordering numbers appear.\n5. Click All Settings to return to the Information and Support pane.\n\nAlternatively, you can also check the ink levels from the printer control panel.'